タイトル

「OpenCVカメラフレーム監視・差分＆ハッシュチェック＋実測FPS測定テスト」

説明

このPythonスクリプトは、PCに接続されたカメラのフレーム取得をリアルタイムで監視し、以下の項目を確認するテスト用ツールです。

主な機能

フレーム間差分チェック

直前フレームとの画素差を計算し、変化があるかどうかを DiffFlag として判定。

画面が静止している場合や同じフレームが連続している場合を検出。

フレームハッシュチェック

グレースケール画像を MD5 ハッシュ化し、前フレームと完全一致かどうかを SameHashFlag として判定。

差分チェックより厳密に「同一フレーム」を検出。

実測FPS計測

フレーム間の経過時間から、直近フレームの FPS と平均FPSを算出。

カメラやキャプチャ処理の性能確認に有用。

フレーム取得間隔計測

各フレーム取得間の時間（ミリ秒）を Interval_sec に記録。

コマ飛びや取得遅延の確認に役立つ。

CSV保存

デスクトップに camera_frame_test.csv として保存。

後から Excel や Pandas で解析可能。

安全な終了

ESC キーまたは Ctrl+C で安全にテストを終了可能。

終了時にカメラリソースを解放。

In [ ]:
import cv2
import time
import hashlib
import numpy as np
import pandas as pd
import os

# === 設定 ===
device_number = 0
target_fps = 25 # カメラ設定FPS
hash_threshold = 5  # 差分の閾値（画面変化の判定）
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
csv_path = os.path.join(desktop_path, "camera_frame_test.csv")

# === カメラ初期化 ===
cap = cv2.VideoCapture(device_number, cv2.CAP_MSMF)
if not cap.isOpened():
    raise RuntimeError("❌ カメラを開けませんでした。")

cap.set(cv2.CAP_PROP_FPS, target_fps)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

print(f"📸 カメラ初期化完了（設定FPS: {target_fps}）")

# --- バッファを最新に ---
for _ in range(10):
    cap.grab()

# === テスト変数 ===
prev_gray = None
prev_hash = None
prev_time = time.time()
start_time = prev_time
frame_id = 0
records = []

# === メインループ ===
try:
    while True:
        # 古いフレームを捨てて最新化 range(*)の*の回数で調整
        # 実ipsを30fpsにするには設定ipsを120にする必要がある。理由は3回分リフレッシュしているから。
        # 15fpsは切り捨てなくても安定している。
        for _ in range(0):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("⚠️ Frame read failed.")
            break

        now = time.time()
        interval = now - prev_time
        prev_time = now

        # 実測FPSを計算（直近の間隔の逆数）
        current_fps = 1.0 / interval if interval > 0 else 0.0

        # グレースケール化して差分
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_hash = hashlib.md5(gray.tobytes()).hexdigest()

        # --- 差分フラグ ---
        if prev_gray is not None:
            diff = cv2.absdiff(gray, prev_gray)
            diff_flag = "〇" if diff.max() > hash_threshold else "×"
        else:
            diff_flag = "〇"

        # --- ハッシュフラグ ---
        if prev_hash is not None:
            same_hash_flag = "〇" if frame_hash == prev_hash else "×"
        else:
            same_hash_flag = "×"

        # --- 結果を記録 ---
        records.append({
            "Frame": frame_id,
            "Interval_sec": interval,
            "FPS": current_fps,
            "DiffFlag": diff_flag,
            "SameHashFlag": same_hash_flag,
            "Timestamp_ms": now * 1000
        })

        # --- コンソール出力 ---
        print(f"[{frame_id:05}] {interval*1000:6.2f} ms | FPS:{current_fps:5.2f} | Diff:{diff_flag} | HashSame:{same_hash_flag}")

        prev_gray = gray
        prev_hash = frame_hash
        frame_id += 1

except KeyboardInterrupt:
    print("⏹ テスト終了（Ctrl+C）")

cap.release()
cv2.destroyAllWindows()

# === 集計 ===
end_time = time.time()
total_time = end_time - start_time
avg_fps = frame_id / total_time if total_time > 0 else 0.0

# === CSV保存 ===
df = pd.DataFrame(records)
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print("\n✅ テスト完了")
print(f"📈 実測平均FPS: {avg_fps:.2f}")
print(f"📁 結果を保存しました: {csv_path}")


↓参考テスト

In [ ]:
# =========================
# スレッドカメラクラスありのテスト→ipsがおかしい？
# =========================
import cv2
import threading
import time
import hashlib
import numpy as np
import pandas as pd
import os

# =========================
# スレッドカメラクラス
# =========================
class CameraCapture:
    """🎥 スレッド化カメラ（常に最新フレームを保持）"""
    def __init__(self, device_number=0, width=720, height=480, target_fps=25):
        api = cv2.CAP_MSMF  # MSMF 固定

        self.cap = cv2.VideoCapture(device_number, api)
        if not self.cap.isOpened():
            raise RuntimeError(f"❌ カメラを開けませんでした (device={device_number}, backend=MSMF)")

        # 解像度・FPS設定
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        self.cap.set(cv2.CAP_PROP_FPS, target_fps)
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # 最新フレームのみ保持

        self.frame = None
        self.lock = threading.Lock()
        self.running = True

        # スレッド開始
        self.thread = threading.Thread(target=self._update, daemon=True)
        self.thread.start()

        # 初期バッファ破棄
        for _ in range(5):
            self.cap.grab()
        time.sleep(0.05)

    def _update(self):
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                time.sleep(0.001)
                continue
            with self.lock:
                self.frame = frame.copy()

    def read(self):
        with self.lock:
            return self.frame.copy() if self.frame is not None else None

    def release(self):
        self.running = False
        self.thread.join(timeout=1)
        self.cap.release()


# =========================
# 設定
# =========================
device_number = 0
target_fps = 25
hash_threshold = 5
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
csv_path = os.path.join(desktop_path, "camera_frame_test_thread.csv")

# =========================
# カメラ初期化
# =========================
cam = CameraCapture(device_number=device_number, target_fps=target_fps)
print(f"📸 スレッドカメラ初期化完了（設定FPS: {target_fps}）")

# =========================
# テスト変数
# =========================
prev_gray = None
prev_hash = None
prev_time = time.time()
start_time = prev_time
frame_id = 0
records = []

# =========================
# メインループ
# =========================
try:
    while True:
        frame = cam.read()
        if frame is None:
            time.sleep(0.001)
            continue

        now = time.time()
        interval = now - prev_time
        prev_time = now
        current_fps = 1.0 / interval if interval > 0 else 0.0

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_hash = hashlib.md5(gray.tobytes()).hexdigest()

        # フレーム差分
        if prev_gray is not None:
            diff = cv2.absdiff(gray, prev_gray)
            diff_flag = "〇" if diff.max() > hash_threshold else "×"
        else:
            diff_flag = "〇"

        # ハッシュチェック
        if prev_hash is not None:
            same_hash_flag = "〇" if frame_hash == prev_hash else "×"
        else:
            same_hash_flag = "×"

        # 記録
        records.append({
            "Frame": frame_id,
            "Interval_sec": interval,
            "FPS": current_fps,
            "DiffFlag": diff_flag,
            "SameHashFlag": same_hash_flag,
            "Timestamp_ms": now * 1000
        })

        # コンソール表示
        print(f"[{frame_id:05}] {interval*1000:6.2f} ms | FPS:{current_fps:5.2f} | Diff:{diff_flag} | HashSame:{same_hash_flag}")

        prev_gray = gray
        prev_hash = frame_hash
        frame_id += 1

        # ESCキーで終了
        if cv2.waitKey(1) & 0xFF == 27:
            print("⏹ ESCキーで終了")
            break

except KeyboardInterrupt:
    print("⏹ Ctrl+C で終了")

finally:
    cam.release()
    cv2.destroyAllWindows()

    # 集計とCSV保存
    total_time = time.time() - start_time
    avg_fps = frame_id / total_time if total_time > 0 else 0.0
    df = pd.DataFrame(records)
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")

    print("\n✅ テスト完了")
    print(f"📈 実測平均FPS: {avg_fps:.2f}")
    print(f"📁 CSV保存先: {csv_path}")


📸 スレッドカメラ初期化完了（設定FPS: 25）
[00000]   0.00 ms | FPS: 0.00 | Diff:〇 | HashSame:×
[00001]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00002]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00003]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00004]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00005]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00006]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00007]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00008]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00009]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00010]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00011]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00012]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00013]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00014]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00015]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00016]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00017]   0.00 ms | FPS: 0.00 | Diff:× | HashSame:〇
[00018]  11.85 ms | FPS:84.41 | Diff:×

In [ ]:
import cv2
import numpy as np
import time

def test_frame_stutter(device_number=0, max_frames=500):
    cap = cv2.VideoCapture(device_number)
    if not cap.isOpened():
        print("カメラを開けません")
        return

    ret, prev_frame = cap.read()
    if not ret:
        print("最初のフレーム取得失敗")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_time = time.time()

    frame_count = 0
    diff_none_frames = []  # 差分なしのフレームを記録
    fps_count = 0
    fps_start = time.time()
    
    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            continue

        # --- フレーム間差分 ---
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray, prev_gray)
        diff_flag = "〇" if diff.max() > 5 else "×"

        # 差分なしなら記録
        if diff_flag == "×":
            diff_none_frames.append((frame_count, time.time()))

        # --- フレーム間時間 ---
        now = time.time()
        interval = now - prev_time
        prev_time = now

        # --- 実測FPS計測 ---
        fps_count += 1
        if now - fps_start >= 1.0:
            print(f"[FPS] {fps_count} fps")
            fps_count = 0
            fps_start = now

        # --- 表示 ---
        display = frame.copy()
        cv2.putText(display, f"Diff: {diff_flag}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(display, f"Interval: {interval*1000:.1f} ms", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Stutter Test", display)

        prev_gray = gray
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

    # --- 差分なしフレームのまとめ表示 ---
    print("差分なしフレーム（フレーム番号, タイムスタンプ）")
    for fnum, ts in diff_none_frames:
        print(f"Frame {fnum} | {ts:.3f} s")

# 使用例
test_frame_stutter(device_number=0, max_frames=500)
